# Scraping and Cleaning NBA dataset

In [1]:
# important imports for data processing
import numpy as np
import pandas as pd

# imports for Web Scraping
from bs4 import BeautifulSoup  # HTML data structure
from urllib.request import urlopen  # Web client

pd.set_option('display.max_columns', None)

## Web Scraping and Data Processing

In [2]:
year = 2020
page_url = "https://www.basketball-reference.com/leagues/NBA_"+str(year)+"_totals.html"

# opens the connection and downloads html page from url
uClient = urlopen(page_url)

# parses html into a soup data structure to traverse html as if it were a json data type.
page_soup = BeautifulSoup(uClient.read(), "html.parser")
uClient.close()

In [3]:
# use findALL() to get the column headers
headerRowHTML = page_soup.findAll('tr', limit=2)

Based on the output below, the 'tr' tag contains the 'th' tag where the name of each column is stored. We're looking at the [0] index because the 'tr' tag applies to every row, not just the header row

In [4]:
# use getText()to extract the text we need into a list
headers = [th.getText() for th in page_soup.findAll('tr', limit=2)[0].findAll('th')]

# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]

Now we're going to run the previous code for every row on the data table (excluding the first which we have as the header already)

In [5]:
playerRowHTML = page_soup.findAll('tr')[1:]

playerRows = [ [val.getText() for val in playerRowHTML[i].findAll('td')] for i in range(len(playerRowHTML)) ] 

In [6]:

basketballDataRaw = pd.DataFrame(playerRows, columns=headers)
basketballDataRaw.describe()

namesBefore = [x for x in basketballDataRaw['Player']]

basketballData = basketballDataRaw
basketballData.describe()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG,FT,FTA,FT,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
count,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624
unique,514,9,22,31,66,65,509,283,390,268,148,265,204,242,340,256,259,189,218,260,124,266,299,222,93,82,152,182,445
top,Jordan McRae,SG,24,TOT,63,0,19,0,7,.000,0,0,.000,0,2,.000,.000,0,0,,0,0,1,0,0,0,0,0,0
freq,4,162,79,54,23,199,5,24,10,19,98,35,63,32,16,20,19,46,40,40,55,16,14,28,50,70,37,20,18


## Cleaning Data (dropping null values, fixing values, etc)

But, we still have to check for missing values and cast number

In [7]:
basketballData.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG,FT,FTA,FT,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Steven Adams,C,26,OKC,58,58,1564,262,443,.591,1,3,.333,261,440,.593,.593,108,183,.590,196,347,543,141,50,65,86,111,633
1,Bam Adebayo,PF,22,MIA,65,65,2235,408,719,.567,1,13,.077,407,706,.576,.568,236,342,.690,165,518,683,333,78,85,185,164,1053
2,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,.493,61,157,.389,330,636,.519,.532,158,191,.827,103,289,392,129,36,87,74,128,1001
3,Nickeil Alexander-Walker,SG,21,NOP,41,0,501,77,227,.339,40,117,.342,37,110,.336,.427,17,28,.607,8,72,80,74,11,7,40,46,211
4,Grayson Allen,SG,24,MEM,30,0,498,79,176,.449,33,91,.363,46,85,.541,.543,30,35,.857,5,61,66,43,6,1,23,36,221


In [8]:
categoricalVals = ['Player', 'Pos', 'Tm']
allVals = basketballData.columns
numericalVals = list(set(allVals) - set(categoricalVals))

print(numericalVals)

for val in numericalVals:
    basketballData[val] = basketballData[val].apply(pd.to_numeric, errors='coerce')


['STL', 'FG', 'FG%', 'FGA', '3P%', '2P', 'TOV', 'G', 'ORB', 'FTA', 'GS', 'BLK', 'DRB', 'FT', 'eFG', '3P', '2PA', 'AST', 'PF', '3PA', 'TRB', 'Age', '2P%', 'PTS', 'MP']


In [9]:
keepPercents = [x for x in headers if '%' in x]
drops = list(set(headers) - set(keepPercents))
print(drops)

['STL', 'FG', 'Tm', 'Player', 'FGA', '2P', 'TOV', 'Pos', 'G', 'ORB', 'FTA', 'GS', 'BLK', 'DRB', 'FT', 'eFG', '3P', '2PA', 'AST', 'PF', '3PA', 'TRB', 'Age', 'PTS', 'MP']


In [10]:
# given the null values present, we must get rid of them (remember earlier function from module 1)

def assess_NA(data):
    # pandas series denoting features and the sum of their null values
    null_sum = data.isnull().sum()# instantiate columns for missing data
    total = null_sum.sort_values(ascending=False)
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False)

    #concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])
    
    return df_NA

print(assess_NA(basketballData))

basketballData.dropna(axis=0, how='any', thresh=None, subset=drops, inplace=True)

        Number of NA  Percent NA
FT                65       10.02
3P%               60        9.24
2P%               37        5.70
FG%               30        4.62
eFG               30        4.62
PTS               25        3.85
Pos               25        3.85
Age               25        3.85
Tm                25        3.85
G                 25        3.85
GS                25        3.85
MP                25        3.85
FG                25        3.85
FGA               25        3.85
3P                25        3.85
3PA               25        3.85
2PA               25        3.85
2P                25        3.85
PF                25        3.85
FT                25        3.85
FTA               25        3.85
ORB               25        3.85
DRB               25        3.85
TRB               25        3.85
AST               25        3.85
STL               25        3.85
BLK               25        3.85
TOV               25        3.85
Player            25        3.85


In [11]:
basketballData.fillna(0, inplace=True)

basketballData.describe()

,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG,FT,FTA,FT,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
count,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000
mean,25.866438,38.607877,17.928082,880.005137,147.542808,321.912671,0.446003,44.313356,124.255137,0.300342,103.229452,197.657534,0.507659,0.510228,63.710616,82.770548,0.743014,36.876712,126.806507,163.683219,87.251712,28.080479,18.190068,50.404110,75.448630,403.109589
std,4.021295,19.976796,22.032682,649.082779,136.983409,292.581004,0.110042,49.027553,128.988804,0.145835,106.369975,199.032124,0.125132,0.109434,78.341362,97.487012,0.155955,40.702601,114.667116,149.950148,100.738474,24.301607,22.427314,48.955937,53.059507,381.679782
min,19.000000,1.000000,0.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,20.750000,0.000000,280.750000,35.750000,80.750000,0.400000,5.000000,17.000000,0.254000,23.000000,44.000000,0.460000,0.475000,11.000000,16.750000,0.667000,10.000000,37.750000,48.000000,19.000000,8.000000,4.000000,14.000000,29.000000,95.500000
50%,25.000000,42.000000,6.000000,779.000000,104.000000,235.500000,0.439000,28.000000,85.000000,0.339000,64.000000,127.000000,0.511000,0.520000,36.000000,51.000000,0.767500,23.500000,96.500000,121.000000,52.000000,22.500000,10.000000,37.000000,72.000000,286.000000
75%,29.000000,57.000000,35.000000,1426.250000,220.000000,494.250000,0.487250,70.000000,194.250000,0.379000,156.000000,287.250000,0.570250,0.558250,82.250000,108.250000,0.835000,52.000000,189.000000,242.750000,112.250000,43.250000,23.250000,72.000000,116.000000,602.000000
max,43.000000,66.000000,65.000000,2243.000000,623.000000,1386.000000,1.000000,271.000000,769.000000,1.000000,540.000000,998.000000,1.000000,1.167000,619.000000,719.000000,1.000000,250.000000,653.000000,869.000000,636.000000,115.000000,187.000000,289.000000,246.000000,2096.000000


In [12]:
basketballData.loc[basketballData['Player']=='Clint Capela']

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG,FT,FTA,FT,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
104,Clint Capela,C,25.0,HOU,39.0,39.0,1279.0,244.0,388.0,0.629,0.0,0.0,0.0,244.0,388.0,0.629,0.629,55.0,104.0,0.529,168.0,369.0,537.0,48.0,33.0,72.0,63.0,102.0,543.0


In [13]:
print(assess_NA(basketballData))

        Number of NA  Percent NA
PTS                0         0.0
2P                 0         0.0
Pos                0         0.0
Age                0         0.0
Tm                 0         0.0
G                  0         0.0
GS                 0         0.0
MP                 0         0.0
FG                 0         0.0
FGA                0         0.0
FG%                0         0.0
3P                 0         0.0
3PA                0         0.0
3P%                0         0.0
2PA                0         0.0
PF                 0         0.0
2P%                0         0.0
eFG                0         0.0
FT                 0         0.0
FTA                0         0.0
FT                 0         0.0
ORB                0         0.0
DRB                0         0.0
TRB                0         0.0
AST                0         0.0
STL                0         0.0
BLK                0         0.0
TOV                0         0.0
Player             0         0.0


In [14]:
#drop duplicate players and keep last stat (most recent team)

basketballData.drop_duplicates(subset='Player', keep='last', inplace=True)

In [15]:
#have to resent indices
basketballData.index = range(0, len(basketballData))

In [16]:
for i in range(len(basketballData['Pos'])):
    basketballData['Pos'][i] = basketballData['Pos'][i][0:2]

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


## Adding data analysis

Looking at the above percentages and players, it's pretty clear that True Shooting Percentage isn't too accurate. It doesn't take into account whether the player sets up their own shot or depends on assists from others. Let's try doing another metric called the Player Efficiency Rating (PER) and graph it to see what happens

In [17]:
playersTS = basketballData['PTS'] / (2 * ( basketballData['FGA'] + 0.44*basketballData['FTA'] ))

basketballData['TS%'] =  playersTS.round(decimals=2)

In [18]:
basketballData.head()
print(list(basketballData.columns))

['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG', 'FT', 'FTA', 'FT', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TS%']


In [19]:
basketballData.columns = ['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TS%']

In [20]:
PER_coeff = {
    "FG": 85.910,
    "STL": 53.897,
    "3P": 51.757,
    "FT": 46.845,
    "BLK": 39.190,
    "ORB": 39.190,
    "AST": 34.677,
    "DRB": 14.707,
    "PF": -17.174,
    "FTmiss": -20.091,
    "FGmiss": -39.190,
    "TOV": -53.897
}

playerER = (
    basketballData["FG"]*PER_coeff["FG"] + 
    basketballData["STL"]*PER_coeff["STL"] + 
    basketballData["3P"]*PER_coeff["3P"] + 
    basketballData["BLK"]*PER_coeff["BLK"] + 
    basketballData["ORB"]*PER_coeff["ORB"] + 
    basketballData["AST"]*PER_coeff["AST"] + 
    basketballData["DRB"]*PER_coeff["DRB"] + 
    basketballData["PF"]*PER_coeff["PF"] + 
    basketballData["FGA"]*PER_coeff["FGmiss"]-
    basketballData["FG"]*PER_coeff["FGmiss"] + 
    basketballData["FTA"]*PER_coeff["FTmiss"]-
    basketballData["FT"]*PER_coeff["FTmiss"] + 
    basketballData["TOV"]*PER_coeff["TOV"]
)

playerER *= (1 / basketballData["MP"])

playerER = playerER.round(decimals=2)

basketballData['PER'] = playerER

Let's try 2 more metrics now, Value added (VA) and Estimated wins added (EWA)

In [21]:
VA_coeff = {
    'PF':11.5,
    'PG':11.0,
    'C':10.6,
    'SF':10.5,
    'SG':10.5
}

PRL = [ VA_coeff[pos] for pos in basketballData['Pos']]

playerVA = (basketballData['MP'] * (basketballData['PER'] - PRL)) / 67
playerVA = playerVA.round(decimals=2)

playerEWA = playerVA/30
playerEWA = playerEWA.round(decimals=1)

basketballData['VA'] = playerVA
basketballData['EWA'] = playerEWA

Adding Advanced Metrics to the Dataset

In [22]:
advanced = pd.read_csv('./data/advanced.csv')

In [23]:
for i in range(len(advanced['Player'])):
    advanced['Player'][i] = advanced['Player'][i].split("\\")[0]
advanced.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Rk,Player,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,1,Steven Adams,14.1,17.2,3.7,2.5,6.2,0.190,2.1,1.1,3.2,2.0
1,2,Bam Adebayo,17.5,20.8,4.6,3.6,8.1,0.175,1.6,2.1,3.6,3.2
2,3,LaMarcus Aldridge,7.8,23.6,3.1,1.4,4.4,0.122,1.8,-0.5,1.3,1.5
3,4,Nickeil Alexander-Walker,14.3,22.8,-0.7,0.4,-0.4,-0.034,-3.5,-1.4,-4.9,-0.4
4,5,Grayson Allen,10.7,17.9,0.5,0.3,0.7,0.070,-1.0,-1.5,-2.6,-0.1


In [24]:
basketballData = basketballData.merge(advanced, on='Player')


In [25]:
basketballData=basketballData.drop('Rk', axis = 1)

In [26]:
basketballData.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,TS%,PER,VA,EWA,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Steven Adams,C,26.0,OKC,58.0,58.0,1564.0,262.0,443.0,0.591,1.0,3.0,0.333,261.0,440.0,0.593,0.593,108.0,183.0,0.590,196.0,347.0,543.0,141.0,50.0,65.0,86.0,111.0,633.0,0.60,19.40,205.42,6.8,14.1,17.2,3.7,2.5,6.2,0.190,2.1,1.1,3.2,2.0
1,Bam Adebayo,PF,22.0,MIA,65.0,65.0,2235.0,408.0,719.0,0.567,1.0,13.0,0.077,407.0,706.0,0.576,0.568,236.0,342.0,0.690,165.0,518.0,683.0,333.0,78.0,85.0,185.0,164.0,1053.0,0.61,18.42,230.84,7.7,17.5,20.8,4.6,3.6,8.1,0.175,1.6,2.1,3.6,3.2
2,LaMarcus Aldridge,C,34.0,SAS,53.0,53.0,1754.0,391.0,793.0,0.493,61.0,157.0,0.389,330.0,636.0,0.519,0.532,158.0,191.0,0.827,103.0,289.0,392.0,129.0,36.0,87.0,74.0,128.0,1001.0,0.57,18.39,203.94,6.8,7.8,23.6,3.1,1.4,4.4,0.122,1.8,-0.5,1.3,1.5
3,Nickeil Alexander-Walker,SG,21.0,NOP,41.0,0.0,501.0,77.0,227.0,0.339,40.0,117.0,0.342,37.0,110.0,0.336,0.427,17.0,28.0,0.607,8.0,72.0,80.0,74.0,11.0,7.0,40.0,46.0,211.0,0.44,8.87,-12.19,-0.4,14.3,22.8,-0.7,0.4,-0.4,-0.034,-3.5,-1.4,-4.9,-0.4
4,Grayson Allen,SG,24.0,MEM,30.0,0.0,498.0,79.0,176.0,0.449,33.0,91.0,0.363,46.0,85.0,0.541,0.543,30.0,35.0,0.857,5.0,61.0,66.0,43.0,6.0,1.0,23.0,36.0,221.0,0.58,11.41,6.76,0.2,10.7,17.9,0.5,0.3,0.7,0.070,-1.0,-1.5,-2.6,-0.1


Now that the data set is complete, we can export it as a CSV to be used in our model file

In [29]:
out_filename = "NBAplayerTotals"+str(year)+".csv"

basketballData.to_csv(out_filename, index=False, header=True)